# Covered Here
* pytest
* tox
* hypothesis


# Introduction pytest

* pytest is a testing framework, including a test runner
* scales nicely from easy first start to testing of complex applications
* don't use `unittest` if you can help it
* only use basic `assert` for asserting
* loads of plugins
* tests all functions/methods prefixed with `test_` in files prefixed `test_`
* run `pytest path` (might be called `py.test`)


some usefuls arguments:

* `-x` stop at first failure
* `-k string` only run tests whose name contains `string`
* `--pdb` jumps into debugger at failed assert

In [ ]:
assert True

In [ ]:
assert False

In [ ]:
assert False, 'Something went Wrong'

In [ ]:
foo = 'bar'
assert foo == 42
assert foo == 45

In [ ]:
cat project/tests/test_simple.py

In [ ]:
!pytest project/tests/test_simple.py

In [ ]:
!cd project
!tox

## fixtures

* use `def test_something(fixture)` for fixtures
* define own fixtures with using `@pytest.fixture` decorator
* see examples in `test_fixtures.py`

In [ ]:
!cat project/tests/test_fixtures.py

## Mocking
* also know as monkeypatching
* uses the `monkeypatch` fixture for monkey patching (also see example

```
def test_something(monkeypatch):
    
    def mockfunction(args):
        return something
    
    monkeypatch.setattr(package.subpackage, 'funcname', mocked_function)
    assert otherfunction_calling_funcname(arguments) == something_else
```

In [ ]:
!cat project/tests/test_mocking.py

<div class="alert alert-success">
    <b>EXERCISE:</b>
        <li> Create some tests for the following capitalize function, including one where you are not passing a string
        <li> Create a fixture that creates a file in a tempdir
        <li> mock the `os.getcwd()` so it always returns the same directory
</div>

In [ ]:
def capitalize(string):
    return string.upper()

# tox
* run your tests in several environments, e.g., different python interpreters, versions of dependencies, etc
* works well with pytest
* detox runs in parallel
* meant to test a package (place `tox.ini` in a directory containing a `setup.py`)

minimal example `tox.ini`
```
[tox]
envlist = {py33,py34,py35,py36,pypy}
skip_missing_interpreters = True

[testenv]
deps =
   pytest
   pytest-cov

commands = 
    py.test {posargs}
```

In [ ]:
!pwd

In [ ]:
!cd project && tox

<div class="alert alert-success">
    <b>EXERCISE:</b>
   
Modify the `tox.ini`, so that the tests get run with two different versions of the timezone database, e.g. `pytz==2017.2` and `pytz==2016.10` get used.
</div>

In [ ]:
[tox]
envlist = {py33,py34,py35,py36}-{pytz201702,pytz201610}
skip_missing_interpreters = True

[testenv]

deps =
    pytest
    pytz201702: pytz==2017.2
    pytz201610: pytz==2016.10

commands =
    py.test  {posargs}


[flake8]
max-line-length = 100
exclude=.tox,examples,doc


# hypothesis
https://hypothesis.readthedocs.io/en/latest/
* property based testing for python
* started out as a quickcheck (Haskell) port for python
* generates test data for your tests
* you can't test for conditions you didn't think of (or can you?)

* use `@given` decorator for specifying data and strategies for generating data
* can use the strategies solo, e.g., see examples
* set fixed examples with `@example`
* change settings with `@settings`
* pytest can give statistics with `--hypothesis-show-statistics`
* build compound data types
* can `.filter()` data and make assumptions with `.assume()` (good, but no magic)

In [ ]:
from hypothesis import given, example, settings, assume
from hypothesis import strategies as st

@given(st.text())
def test_upper_lower(s):
    assert s.upper().lower() == s.lower()

In [ ]:
test_upper_lower()

In [ ]:
text = st.text()
text.example()

In [ ]:
@given(st.text())
@example('ß')
def test_upper_lower(s):
    assert s.upper().lower() == s.lower()


In [ ]:
test_upper_lower()

In [ ]:
@given(st.text())
@settings(max_examples=2000)
def test_upper_lower(s):
    assert s.upper().lower() == s.lower()

In [ ]:
test_upper_lower()

In [ ]:
@given(st.tuples(st.booleans(), st.text()))
def test_look_tuples_work_too(t):
    # A tuple is generated as the one you provided, with the corresponding
    # types in those positions.
    assert len(t) == 2
    assert isinstance(t[0], bool)
    assert isinstance(t[1], str)
    
test_look_tuples_work_too()

In [ ]:
@given(st.integers().filter(lambda x: x % 2 == 0))
def test_even_integers(i):
    assert i % 2 == 0
    
test_even_integers()

In [ ]:
@given(st.lists(st.integers()))
def test_sum_is_positive(xs):
    assume(len(xs) > 3)
    assume(all(x > 0 for x in xs))
    print(xs)
    assert sum(xs) > 0

test_sum_is_positive()

* support for numpy arrays with `hypothesis.extra.numpy'

In [ ]:
from hypothesis.extra.numpy import arrays
import numpy as np

In [ ]:
arrays(np.int8, (2, 3)).example()

In [ ]:
arrays(np.float64, (8), elements=st.floats(min_value=0, max_value=1)).example()

In [ ]:
def rnd_len_arrays(dtype, min_len=1, max_len=3, elements=None):
    lengths = st.integers(min_value=min_len, max_value=max_len)
    return lengths.flatmap(lambda n: arrays(dtype, n, elements=elements))

In [ ]:
rnd_len_arrays(np.int16).example()